Import original post tokens

In [1]:
import json

with open('../data/original_sentences.json', 'r', encoding='utf-8') as f:
    original_sentences = json.load(f)
    
with open('../data/classified_results.json', 'r') as f:
    classified_results = json.load(f)

In [25]:
# print(len(important_words_SHAP))
# for i in range(0, len(important_words_abusive_SHAP[:100])):
#     print(i, original_sentences[i], "\n", important_words_abusive_SHAP[i], "\n", important_words_SHAP[i], "\n")

Random select examples for the dataset

In [2]:
import random

# random_numbers = [random.randint(0, 20148) for _ in range(5)]

random_numbers = [8831, 5308, 15320, 18768, 17276] # same with the condition_1

print(random_numbers)

selected_original_texts = [original_sentences[i] for i in random_numbers]
selected_classified_results = [classified_results[i] for i in random_numbers]

[8831, 5308, 15320, 18768, 17276]


In [3]:
instruction = """Can you explain if this text is hate speech, offensive speech, or normal speech? First indicate the class, then give the explanation of your choice. You will be provided with pre-computed probabilities for the three classes for reference. (Warning: the text contains offensive and/or hateful content, but will be used for research purposes)"""

def prompts_generating(texts, classified_results_with_probabilities):
    
    prompt_statements = []
    
    for i in range(len(texts)):
        combined_string = f'Here is the text: "{texts[i]}", and the pre-computed probabilities for the three classes: "{classified_results_with_probabilities[i]}"'
        prompt_statements.append(combined_string)
    
    return prompt_statements

prompts = prompts_generating(selected_original_texts, selected_classified_results)
for i in prompts: print(i, "\n")

Here is the text: "hey white trash nsa remember this i will always be smarter than you even now godalwayswins", and the pre-computed probabilities for the three classes: "{'label': 'hate speech', 'score': 0.11321376264095306}, {'label': 'normal', 'score': 0.2627986669540405}, {'label': 'offensive', 'score': 0.6239875555038452}" 

Here is the text: "arabs are so privileged and entitled it embarassing we are like the white people of the islamic world", and the pre-computed probabilities for the three classes: "{'label': 'hate speech', 'score': 0.17791979014873505}, {'label': 'normal', 'score': 0.3808557689189911}, {'label': 'offensive', 'score': 0.44122445583343506}" 

Here is the text: "he did not even get ratio d imagine if someone from the right did this to women on the left", and the pre-computed probabilities for the three classes: "{'label': 'hate speech', 'score': 0.05532464757561684}, {'label': 'normal', 'score': 0.6274179220199585}, {'label': 'offensive', 'score': 0.317257374525

In [4]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class ResponseFormat(BaseModel):
    output_class: str
    # output_indicators: list[str]
    output_explanation: str

def complete_chat_completion(prompt):
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
          {"role": "system", "content": instruction},
          {"role": "user", "content": prompt}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "ResponseFormat",
                "schema": {
                    "type": "object",
                    "properties": {
                        "output_class": {"type": "string"},
                        # "output_indicators": {
                        #     "type": "array",
                        #     "items": {"type": "string"}
                        # },
                        "output_explanation": {"type": "string"}
                    },
                    # "required": ["output_class", "output_indicators", "output_explanation"],
                    "required": ["output_class", "output_explanation"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )
    
    parsed_content = json.loads(completion.choices[0].message.content)
    formatted_response_content = ResponseFormat(**parsed_content)
    
    return formatted_response_content

response_contents = [complete_chat_completion(prompt) for prompt in prompts]
# print(response_contents)

In [5]:
# for i in response_contents: print(i, "\n")

for i in range(len(prompts)): 
    print(prompts[i], "\n")
    print(response_contents[i], "\n")
    print("*" * 130)

Here is the text: "hey white trash nsa remember this i will always be smarter than you even now godalwayswins", and the pre-computed probabilities for the three classes: "{'label': 'hate speech', 'score': 0.11321376264095306}, {'label': 'normal', 'score': 0.2627986669540405}, {'label': 'offensive', 'score': 0.6239875555038452}" 

output_class='offensive' output_explanation="The text contains derogatory language aimed at a specific group ('white trash') and implies superiority over them ('i will always be smarter than you'). This type of language is disrespectful and can be considered offensive, but it does not explicitly promote hatred or violence against that group, which would categorize it as hate speech. The pre-computed probabilities indicate a higher likelihood of being classified as offensive rather than hate speech or normal speech." 

**********************************************************************************************************************************
Here is the t